<a href="https://colab.research.google.com/github/varshanj-hub/Fine-Tuning-DeepSeek-with-Custom-Dataset/blob/main/Fine_Tuning_DS_with_Custom_Data_using_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers datasets peft accelerate bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import json
import wandb

# Initialize WandB
wandb.init(project="RE-Maintance-QLoRA")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: varshanj028 (varshanj028-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
model = prepare_model_for_kbit_training(model)

# LoRA Config (optimized for T4 GPU)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [6]:
import json

samples = [
    {
        "prompt": "Question 1: How often should you change the engine oil in a Royal Enfield Classic 350?",
        "completion": "Answer 1: It's recommended to change the engine oil every 3,000–5,000 km depending on riding conditions."
    },
    {
        "prompt": "Question 2: What type of engine oil is best for a Royal Enfield Himalayan?",
        "completion": "Answer 2: A semi-synthetic or fully synthetic 15W50 or 20W50 grade oil is recommended for the Himalayan."
    },
    {
        "prompt": "Question 3: How do you adjust the clutch cable on a Royal Enfield Bullet 500?",
        "completion": "Answer 3: Loosen the locknut, adjust the cable tension using the adjuster near the lever, then tighten the locknut while maintaining 10-15mm free play."
    },
    {
        "prompt": "Question 4: What is the ideal tyre pressure for a Royal Enfield Interceptor 650?",
        "completion": "Answer 4: The recommended tyre pressure is 32 PSI (front) and 36 PSI (rear) for solo riding with normal load."
    },
    {
        "prompt": "Question 5: How often should you clean and lubricate the chain on a Royal Enfield?",
        "completion": "Answer 5: Clean and lubricate the chain every 500 km or after riding in wet/muddy conditions."
    },
    {
        "prompt": "Question 6: What is the recommended valve clearance for a Royal Enfield Classic 500?",
        "completion": "Answer 6: The ideal valve clearance is 0.08–0.10 mm for intake and 0.13–0.15 mm for exhaust."
    },
    {
        "prompt": "Question 7: How do you fix a fuel leak in a Royal Enfield Thunderbird?",
        "completion": "Answer 7: Check the fuel hose, carburetor gasket, and fuel tap for leaks. Replace damaged parts and ensure tight connections."
    },
    {
        "prompt": "Question 8: What causes excessive engine heating in a Royal Enfield?",
        "completion": "Answer 8: Possible causes include low engine oil, incorrect fuel-air mixture, clogged cooling fins, or prolonged idling."
    },
    {
        "prompt": "Question 9: How often should you replace the air filter in a Royal Enfield?",
        "completion": "Answer 9: Replace the air filter every 5,000–10,000 km, or sooner if riding in dusty conditions."
    },
    {
        "prompt": "Question 10: What is the correct battery voltage for a Royal Enfield?",
        "completion": "Answer 10: A healthy battery should read 12.6–13.2V when idle and 13.5–14.5V when the engine is running."
    },
    {
        "prompt": "Question 11: How do you troubleshoot a Royal Enfield that won't start?",
        "completion": "Answer 11: Check the battery, spark plug, fuel supply, ignition fuse, and kickstand kill switch."
    },
    {
        "prompt": "Question 12: What is the recommended spark plug gap for a Royal Enfield Bullet 350?",
        "completion": "Answer 12: The ideal spark plug gap is 0.6–0.7 mm for most Royal Enfield models."
    },
    {
        "prompt": "Question 13: How do you adjust the headlight beam on a Royal Enfield?",
        "completion": "Answer 13: Use the adjuster screws near the headlight assembly to align the beam height and direction."
    },
    {
        "prompt": "Question 14: What is the best way to clean Royal Enfield’s chrome parts?",
        "completion": "Answer 14: Use a mild soap solution and a soft cloth, then dry and apply chrome polish to prevent rust."
    },
    {
        "prompt": "Question 15: How often should you replace the brake fluid in a Royal Enfield?",
        "completion": "Answer 15: Replace brake fluid every 2 years or 20,000 km, whichever comes first."
    },
    {
        "prompt": "Question 16: What causes a Royal Enfield to backfire?",
        "completion": "Answer 16: Common causes include a lean fuel mixture, exhaust leaks, incorrect ignition timing, or a faulty spark plug."
    },
    {
        "prompt": "Question 17: How do you check and top up fork oil in a Royal Enfield?",
        "completion": "Answer 17: Remove the fork cap, drain old oil, refill with recommended fork oil (usually 10W or 15W), and measure the correct level."
    },
    {
        "prompt": "Question 18: What is the proper way to store a Royal Enfield for long periods?",
        "completion": "Answer 18: Fill the tank, add fuel stabilizer, disconnect the battery, inflate tyres, and store in a dry place."
    },
    {
        "prompt": "Question 19: How do you align the front wheel of a Royal Enfield?",
        "completion": "Answer 19: Loosen the axle nut, adjust the alignment using the fork markings, and ensure equal gap on both sides."
    },
    {
        "prompt": "Question 20: What is the best chain lubricant for a Royal Enfield?",
        "completion": "Answer 20: Use a high-quality motorcycle chain lube like Motul Chain Lube or SAE 90 gear oil."
    },
    {
        "prompt": "Question 21: How do you diagnose a misfiring Royal Enfield engine?",
        "completion": "Answer 21: Check spark plugs, ignition coil, fuel quality, carburetor tuning, and compression levels."
    },
    {
        "prompt": "Question 22: What is the recommended idle RPM for a Royal Enfield Classic 350?",
        "completion": "Answer 22: The ideal idle RPM is between 1,000–1,200 RPM when the engine is warm."
    },
    {
        "prompt": "Question 23: How do you clean a clogged carburetor on a Royal Enfield?",
        "completion": "Answer 23: Disassemble the carburetor, clean jets with carb cleaner, blow compressed air, and reassemble with new gaskets if needed."
    },
    {
        "prompt": "Question 24: What is the torque setting for Royal Enfield’s wheel nuts?",
        "completion": "Answer 24: The recommended torque is 45–50 Nm for front and rear wheel nuts."
    },
    {
        "prompt": "Question 25: How often should you check the spoke tension on Royal Enfield wheels?",
        "completion": "Answer 25: Inspect spoke tension every 5,000 km or after off-road rides."
    },
    {
        "prompt": "Question 26: What causes oil leakage from the Royal Enfield engine head?",
        "completion": "Answer 26: Worn-out gaskets, loose bolts, or a warped cylinder head can cause oil leaks."
    },
    {
        "prompt": "Question 27: How do you adjust the rear suspension on a Royal Enfield Himalayan?",
        "completion": "Answer 27: Use the preload adjuster ring to increase/decrease stiffness based on load and riding style."
    },
    {
        "prompt": "Question 28: What is the best coolant for a Royal Enfield Interceptor 650?",
        "completion": "Answer 28: Use a high-quality ethylene glycol-based coolant (50:50 mix with distilled water)."
    },
    {
        "prompt": "Question 29: How do you fix a slipping clutch on a Royal Enfield?",
        "completion": "Answer 29: Adjust clutch cable free play, check clutch plates for wear, and replace if necessary."
    },
    {
        "prompt": "Question 30: What is the recommended tyre size for a Royal Enfield Bullet 500?",
        "completion": "Answer 30: Standard sizes are 3.25x19 (front) and 3.5x19 (rear) for most Bullet 500 models."
    },
    {
        "prompt": "Question 31: How do you bleed brakes on a Royal Enfield?",
        "completion": "Answer 31: Attach a bleed tube to the caliper nipple, open the nipple, pump the brake lever, and close the nipple while keeping the reservoir filled."
    },
    {
        "prompt": "Question 32: What causes excessive vibration in a Royal Enfield?",
        "completion": "Answer 32: Loose engine mounts, unbalanced wheels, worn-out sprockets, or misaligned chain can cause vibrations."
    },
    {
        "prompt": "Question 33: How do you replace the throttle cable on a Royal Enfield?",
        "completion": "Answer 33: Remove old cable, route the new one along the same path, adjust free play (2–3mm), and secure connections."
    },
    {
        "prompt": "Question 34: What is the best way to prevent rust on a Royal Enfield’s exhaust?",
        "completion": "Answer 34: Apply high-temperature exhaust paint or anti-rust spray after cleaning with a wire brush."
    },
    {
        "prompt": "Question 35: How often should you replace the brake pads on a Royal Enfield?",
        "completion": "Answer 35: Replace brake pads when thickness is below 2mm or every 15,000–20,000 km."
    },
    {
        "prompt": "Question 36: What is the correct way to jump-start a Royal Enfield?",
        "completion": "Answer 36: Connect jumper cables (red to +, black to – on donor battery), start the donor bike, then start the Enfield."
    },
    {
        "prompt": "Question 37: How do you adjust the tappets on a Royal Enfield?",
        "completion": "Answer 37: Remove the tappet cover, rotate the engine to TDC, adjust clearance with feeler gauges, and lock the adjuster nut."
    },
    {
        "prompt": "Question 38: What causes a Royal Enfield to lose power uphill?",
        "completion": "Answer 38: Clogged air filter, weak spark, incorrect fuel mixture, or carbon buildup in the exhaust can reduce power."
    },
    {
        "prompt": "Question 39: How do you check the electrical wiring on a Royal Enfield?",
        "completion": "Answer 39: Inspect for frayed wires, loose connections, and test circuits with a multimeter."
    },
    {
        "prompt": "Question 40: What is the best way to clean a Royal Enfield’s fuel tank?",
        "completion": "Answer 40: Drain fuel, rinse with vinegar or a fuel tank cleaner, scrub rust with a chain, and dry thoroughly."
    },
    {
        "prompt": "Question 41: How do you replace the rear shock absorbers on a Royal Enfield?",
        "completion": "Answer 41: Remove the mounting bolts, swap the old shocks with new ones, and torque bolts to spec."
    },
    {
        "prompt": "Question 42: What is the recommended service interval for a Royal Enfield?",
        "completion": "Answer 42: Follow the first service at 500 km, then every 5,000 km or as per the manual."
    },
    {
        "prompt": "Question 43: How do you fix a sticky throttle on a Royal Enfield?",
        "completion": "Answer 43: Clean and lubricate the throttle cable, check grip alignment, and ensure smooth movement."
    },
    {
        "prompt": "Question 44: What is the best way to break in a new Royal Enfield engine?",
        "completion": "Answer 44: Avoid high RPMs for the first 1,000 km, vary speeds, and change oil after the break-in period."
    },
    {
        "prompt": "Question 45: How do you adjust the headlight focus on a Royal Enfield?",
        "completion": "Answer 45: Use the adjuster screws behind the headlight to tilt it up/down or left/right."
    },
    {
        "prompt": "Question 46: What causes a Royal Enfield’s battery to drain quickly?",
        "completion": "Answer 46: Faulty regulator-rectifier, parasitic drain, old battery, or loose connections can cause rapid drainage."
    },
    {
        "prompt": "Question 47: How do you replace the clutch plates on a Royal Enfield?",
        "completion": "Answer 47: Drain oil, remove clutch cover, extract old plates, install new ones, and adjust cable tension."
    },
    {
        "prompt": "Question 48: What is the best way to clean a Royal Enfield’s engine?",
        "completion": "Answer 48: Use a degreaser, soft brush, and water (avoid direct spray on electrical parts)."
    },
    {
        "prompt": "Question 49: How do you check the compression on a Royal Enfield engine?",
        "completion": "Answer 49: Use a compression tester; healthy engines should show 120–150 PSI."
    },
    {
        "prompt": "Question 50: What is the best tyre for off-road use on a Royal Enfield Himalayan?",
        "completion": "Answer 50: CEAT Gripp or Metzeler Tourance are popular off-road tyre choices."
    },
    # Continuing to 100...
    {
        "prompt": "Question 51: How do you fix a Royal Enfield’s speedometer if it stops working?",
        "completion": "Answer 51: Check the speed sensor cable connection, fuse, and instrument cluster wiring."
    },
    {
        "prompt": "Question 52: What is the best way to prevent a Royal Enfield’s chain from rusting?",
        "completion": "Answer 52: Clean and lubricate regularly, avoid water exposure, and use a rust-inhibiting chain lube."
    },
    {
        "prompt": "Question 53: How do you replace the front fork seals on a Royal Enfield?",
        "completion": "Answer 53: Remove the fork tubes, extract old seals, clean the area, and press in new seals with a driver tool."
    },
    {
        "prompt": "Question 54: What is the best way to remove scratches from a Royal Enfield’s paint?",
        "completion": "Answer 54: Use rubbing compound for light scratches or touch-up paint for deeper ones."
    },
    {
        "prompt": "Question 55: How do you adjust the rear brake pedal height on a Royal Enfield?",
        "completion": "Answer 55: Loosen the locknut on the brake rod and adjust the length until the pedal sits comfortably."
    },
    {
        "prompt": "Question 56: What causes a Royal Enfield’s exhaust to turn blue?",
        "completion": "Answer 56: Overheating or oil burning in the exhaust can cause bluing."
    },
    {
        "prompt": "Question 57: How do you replace the handlebar grips on a Royal Enfield?",
        "completion": "Answer 57: Use compressed air or soapy water to slide off old grips; apply grip glue for new ones."
    },
    {
        "prompt": "Question 58: What is the best way to store a Royal Enfield battery?",
        "completion": "Answer 58: Keep it fully charged, disconnect terminals, and store in a cool, dry place."
    },
    {
        "prompt": "Question 59: How do you check the wheel bearings on a Royal Enfield?",
        "completion": "Answer 59: Lift the wheel and check for play or roughness when spinning."
    },
    {
        "prompt": "Question 60: What is the best way to clean a Royal Enfield’s air filter?",
        "completion": "Answer 60: For foam filters, wash with mild soap, dry, and apply filter oil."
    },
    {
        "prompt": "Question 61: How do you fix a Royal Enfield’s horn if it stops working?",
        "completion": "Answer 61: Check the fuse, wiring, and horn relay; clean contacts if corroded."
    },
    {
        "prompt": "Question 62: What is the best way to prevent a Royal Enfield’s fuel tank from rusting?",
        "completion": "Answer 62: Keep the tank full when storing, use fuel stabilizer, and apply anti-rust coatings."
    },
    {
        "prompt": "Question 63: How do you replace the footpegs on a Royal Enfield?",
        "completion": "Answer 63: Remove the mounting bolt, swap old pegs with new ones, and tighten securely."
    },
    {
        "prompt": "Question 64: What causes a Royal Enfield’s gearbox to make noise?",
        "completion": "Answer 64: Low oil, worn gears, or improper clutch adjustment can cause gearbox noise."
    },
    {
        "prompt": "Question 65: How do you adjust the side stand on a Royal Enfield?",
        "completion": "Answer 65: Bend the stand slightly or adjust the mounting bolt for better alignment."
    },
    {
        "prompt": "Question 66: What is the best way to clean a Royal Enfield’s carburetor?",
        "completion": "Answer 66: Use carb cleaner spray, compressed air, and a soft brush for jets."
    },
    {
        "prompt": "Question 67: How do you replace the rearview mirrors on a Royal Enfield?",
        "completion": "Answer 67: Unscrew the old mirror, align the new one, and tighten the mounting nut."
    },
    {
        "prompt": "Question 68: What is the best way to remove dents from a Royal Enfield’s fuel tank?",
        "completion": "Answer 68: Use a dent puller kit or seek professional paintless dent removal."
    },
    {
        "prompt": "Question 69: How do you check the steering bearings on a Royal Enfield?",
        "completion": "Answer 69: Lift the front wheel and check for smooth movement without play."
    },
    {
        "prompt": "Question 70: What is the best way to waterproof a Royal Enfield’s electrical system?",
        "completion": "Answer 70: Use dielectric grease on connectors and ensure proper sealing of components."
    },
    {
        "prompt": "Question 71: How do you replace the turn signals on a Royal Enfield?",
        "completion": "Answer 71: Disconnect wiring, unscrew the old unit, and install the new one."
    },
    {
        "prompt": "Question 72: What causes a Royal Enfield’s engine to smoke?",
        "completion": "Answer 72: Worn piston rings (blue smoke), excess oil (white smoke), or coolant leak (white smoke)."
    },
    {
        "prompt": "Question 73: How do you adjust the brake light switch on a Royal Enfield?",
        "completion": "Answer 73: Loosen the locknut and adjust the switch position for proper activation."
    },
    {
        "prompt": "Question 74: What is the best way to clean a Royal Enfield’s speedometer cable?",
        "completion": "Answer 74: Remove, lubricate with grease, and reinstall."
    },
    {
        "prompt": "Question 75: How do you replace the kickstart lever on a Royal Enfield?",
        "completion": "Answer 75: Remove the pinch bolt, slide off the old lever, and install the new one."
    },
    {
        "prompt": "Question 76: What is the best way to fix a Royal Enfield’s loose handlebar?",
        "completion": "Answer 76: Tighten the handlebar clamp bolts to the recommended torque."
    },
    {
        "prompt": "Question 77: How do you replace the seat on a Royal Enfield?",
        "completion": "Answer 77: Unlock or unscrew the seat latch, remove the old seat, and install the new one."
    },
    {
        "prompt": "Question 78: What causes a Royal Enfield’s clutch to feel hard?",
        "completion": "Answer 78: Dry cable, misalignment, or worn clutch plates can cause stiffness."
    },
    {
        "prompt": "Question 79: How do you adjust the neutral light sensor on a Royal Enfield?",
        "completion": "Answer 79: Check wiring and adjust the sensor near the gearbox."
    },
    {
        "prompt": "Question 80: What is the best way to clean a Royal Enfield’s fuel injectors?",
        "completion": "Answer 80: Use a fuel injector cleaner additive or ultrasonic cleaning."
    },
    {
        "prompt": "Question 81: How do you replace the side cover on a Royal Enfield?",
        "completion": "Answer 81: Remove screws or clips, align the new cover, and secure."
    },
    {
        "prompt": "Question 82: What causes a Royal Enfield’s headlight to flicker?",
        "completion": "Answer 82: Loose wiring, faulty regulator, or weak battery can cause flickering."
    },
    {
        "prompt": "Question 83: How do you adjust the rear brake light on a Royal Enfield?",
        "completion": "Answer 83: Adjust the brake light switch near the pedal."
    },
    {
        "prompt": "Question 84: What is the best way to fix a Royal Enfield’s stuck fuel cap?",
        "completion": "Answer 84: Lubricate the lock mechanism with graphite powder."
    },
    {
        "prompt": "Question 85: How do you replace the gear shifter on a Royal Enfield?",
        "completion": "Answer 85: Remove the linkage bolt, swap the lever, and tighten."
    },
    {
        "prompt": "Question 86: What causes a Royal Enfield’s engine to stall at idle?",
        "completion": "Answer 86: Dirty carburetor, incorrect idle RPM, or vacuum leaks can cause stalling."
    },
    {
        "prompt": "Question 87: How do you adjust the front brake lever on a Royal Enfield?",
        "completion": "Answer 87: Adjust the reach screw near the lever for comfort."
    },
    {
        "prompt": "Question 88: What is the best way to clean a Royal Enfield’s brake calipers?",
        "completion": "Answer 88: Remove, clean with brake cleaner, and lubricate slider pins."
    },
    {
        "prompt": "Question 89: How do you replace the rear sprocket on a Royal Enfield?",
        "completion": "Answer 89: Remove the wheel, unbolt the sprocket, and install the new one."
    },
    {
        "prompt": "Question 90: What causes a Royal Enfield’s chain to keep loosening?",
        "completion": "Answer 90: Worn sprockets, improper tension, or weak chain can cause loosening."
    },
    {
        "prompt": "Question 91: How do you adjust the pilot screw on a Royal Enfield carburetor?",
        "completion": "Answer 91: Turn the screw 1.5–2 turns out from fully closed as a baseline."
    },
    {
        "prompt": "Question 92: What is the best way to fix a Royal Enfield’s leaking fork seals?",
        "completion": "Answer 92: Replace the seals and clean the fork tubes."
    },
    {
        "prompt": "Question 93: How do you replace the ignition coil on a Royal Enfield?",
        "completion": "Answer 93: Disconnect wiring, unbolt the old coil, and install the new one."
    },
    {
        "prompt": "Question 94: What causes a Royal Enfield’s engine to knock?",
        "completion": "Answer 94: Low-octane fuel, carbon buildup, or worn bearings can cause knocking."
    },
    {
        "prompt": "Question 95: How do you adjust the choke cable on a Royal Enfield?",
        "completion": "Answer 95: Loosen the locknut, adjust cable slack, and retighten."
    },
    {
        "prompt": "Question 96: What is the best way to clean a Royal Enfield’s spark arrestor?",
        "completion": "Answer 96: Remove and burn off carbon deposits with a torch."
    },
    {
        "prompt": "Question 97: How do you replace the side stand spring on a Royal Enfield?",
        "completion": "Answer 97: Use pliers to hook the new spring into place."
    },
    {
        "prompt": "Question 98: What causes a Royal Enfield’s speedometer cable to break?",
        "completion": "Answer 98: Lack of lubrication or kinking can cause cable failure."
    },
    {
        "prompt": "Question 99: How do you adjust the tachometer on a Royal Enfield?",
        "completion": "Answer 99: Most are non-adjustable; check wiring if faulty."
    },
    {
        "prompt": "Question 100: What is the best way to winterize a Royal Enfield?",
        "completion": "Answer 100: Change oil, add fuel stabilizer, inflate tires, and store on a stand."
    }
]

In [8]:
import json

# Write the niche samples to a JSONL file.
with open("niche_dataset.jsonl", "w", encoding="utf-8") as f:
    for sample in samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + "\n")

print("niche_dataset.jsonl has been created with 100 Royal ENfield Maintance samples.")

niche_dataset.jsonl has been created with 100 Royal ENfield Maintance samples.


In [9]:
# Load dataset (replace with your JSONL path)
with open("niche_dataset.jsonl", "r", encoding="utf-8") as f:
    samples = [json.loads(line) for line in f]

dataset = Dataset.from_list(samples)
train_test_split = dataset.train_test_split(test_size=0.2)

# Tokenization function
def tokenize_function(examples):
    combined_texts = [f"{p}\n{c}" for p, c in zip(examples["prompt"], examples["completion"])]
    tokenized = tokenizer(
        combined_texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

tokenized_train = train_test_split["train"].map(tokenize_function, batched=True)
tokenized_eval = train_test_split["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [13]:
training_args = TrainingArguments(
    output_dir="./deepseek_qlora_finetuned",
    per_device_train_batch_size=2,   # Critical for T4 memory limits
    gradient_accumulation_steps=8,   # Simulates larger batch size
    num_train_epochs=30,
    learning_rate=2e-5,
    fp16=True,                       # Mixed precision
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    report_to="wandb",
    optim="paged_adamw_8bit",        # Prevents OOM errors
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two va

Step,Training Loss,Validation Loss
50,2.043900,1.475056
100,0.437200,0.381464
150,0.380100,0.367142


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=150, training_loss=1.821378812789917, metrics={'train_runtime': 1179.8057, 'train_samples_per_second': 2.034, 'train_steps_per_second': 0.127, 'total_flos': 1.1389529751552e+16, 'train_loss': 1.821378812789917, 'epoch': 30.0})

In [15]:
# Define save path in Google Drive
save_path = "/content/drive/MyDrive/deepseek_qlora_finetuned"

# Save only the LoRA adapter (small file)
model.save_pretrained(save_path)

# Save tokenizer
tokenizer.save_pretrained(save_path)

# Verify
print(f"Model and tokenizer saved to: {save_path}")
print(f"Adapter size: {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters")

Model and tokenizer saved to: /content/drive/MyDrive/deepseek_qlora_finetuned
Adapter size: 1089536 trainable parameters


In [16]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Reload base model in 4-bit (to match QLoRA training)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Same config used earlier
    device_map="auto"
)

In [17]:
# Load fine-tuned adapter
model = PeftModel.from_pretrained(
    base_model,
    save_path,                      # Path where adapter was saved
    torch_dtype=torch.bfloat16       # Match training precision
)

# Merge and unload (de-quantizes temporarily)
merged_model = model.merge_and_unload()  # Now a full-precision model

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [18]:
final_save_path = "/content/drive/MyDrive/deepseek_finetuned_full"

# Save full merged model (will be LARGE ~3-6GB)
merged_model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)

print(f"Full merged model saved to: {final_save_path}")

Full merged model saved to: /content/drive/MyDrive/deepseek_finetuned_full


In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define path
final_save_path = "/content/drive/MyDrive/deepseek_finetuned_full"

# Load merged model (full precision)
model = AutoModelForCausalLM.from_pretrained(
    final_save_path,
    device_map="auto"  # Automatically places layers on GPU/CPU
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

print(f"Successfully loaded model and tokenizer from {final_save_path}")

Successfully loaded model and tokenizer from /content/drive/MyDrive/deepseek_finetuned_full


In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [30]:
# Define path
model_path = "/content/drive/MyDrive/deepseek_finetuned_full"

# Load merged model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [34]:
from transformers import pipeline
import re

# Initialize pipeline (do this once)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    truncation=True
)

# Single prompt input - modify this line only for different questions
prompt = "What is the average time duration to change the engine oil in a Royal Enfield?"

# Generate and clean output
response = pipe(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    temperature=0.7,
    repetition_penalty=1.2
)[0]['generated_text']

# Remove prompt and any repetitions
clean_response = re.sub(rf'{re.escape(prompt)}|(I need to know.*?\.)', '', response).strip()

print(clean_response)

Device set to use cuda:0


Please provide your answer according to the... (I need more information) What do you think?
The average time duration of changing an engine oil depends on several factors, including how often they are changed. If it's more frequent than usual, then people might be putting more effort into it or perhaps not stopping for long periods when they're driving.
There could also be other variables like the location where they live and whether there are traffic lights or similar obstacles that can slow down their drive.

Additionally,
